# See if I can log multi-game logs!

Note: This is just a sanity check to ensure that I can load up JSONs to study them, not a proper evaluation.

In [1]:
import os
import sys
import json
from pandas import DataFrame, json_normalize
from typing import List, Dict, Any, Tuple, Union, Optional

LOGS_PATH: str = "../expt-logs/"

In [2]:
EXPT_NAME: str = "2025-01-24_exp_2"

In [3]:
agent_logs_path: str = os.path.join(LOGS_PATH, EXPT_NAME + "/agent-logs.json")

In [4]:
def read_jsonl_as_json(file_path):
    with open(file_path, 'r') as file:
        return [json.loads(line) for line in file]

In [7]:
def load_agent_logs_df(path: str) -> DataFrame:
    df: DataFrame = json_normalize(
        read_jsonl_as_json(path),
        meta=[
            "timestamp",
            ["player", "name"],
            ["player", "identity"],
            ["interaction", "type"],
            ["interaction", "response", "Condensed Memory"],
            ["interaction", "response", "Thinking Process", "thought"],
            ["interaction", "response", "Thinking Process", "action"],
        ],
    )
    df["action"] = df["interaction.response.Thinking Process.action"].combine_first(
        df["interaction.response.Action"]
    )
    df = df.drop(
        columns=[
            "interaction.response.Thinking Process.action",
            "interaction.response.Action",
        ]
    )

    # same, merge 'interaction.response.Thinking Process' with 'interaction.response.Thinking Process.thought' into 'thought'
    df["thought"] = df["interaction.response.Thinking Process.thought"].combine_first(
        df["interaction.response.Thinking Process"]
    )
    df = df.drop(
        columns=[
            "interaction.response.Thinking Process.thought",
            "interaction.response.Thinking Process",
        ]
    )
    return df

In [8]:
df: DataFrame = load_agent_logs_df(agent_logs_path)

In [10]:
df

,game_index,step,timestamp,player.name,player.identity,interaction.prompt.summarization,interaction.prompt.all_info,interaction.prompt.memory,interaction.response.Condensed Memory,action,thought
0,Game 1,0,2025-01-24 21:06:28.753606,Player 1: blue,Impostor,no thought process has been made,Game Time: 0/50\nCurrent phase: Task phase\nIn...,no memory has been processed,"The game has just started with all players, in...",MOVE from Cafeteria to Medbay,Given that the game is in the task phase and t...
1,Game 1,0,2025-01-24 21:06:32.110222,Player 2: purple,Crewmate,no thought process has been made,Game Time: 0/50\nCurrent phase: Task phase\nIn...,no memory has been processed,"- No significant past events to recall, as thi...",MOVE from Cafeteria to Upper Engine,"- It is the beginning of the Task phase, meani..."
2,Game 1,0,2025-01-24 21:06:35.674405,Player 3: brown,Crewmate,no thought process has been made,Game Time: 0/50\nCurrent phase: Task phase\nIn...,no memory has been processed,- Players 1 (blue) and 2 (purple) have left th...,MOVE from Cafeteria to Upper Engine,"- As a Crewmate, my primary objective is to co..."
3,Game 1,0,2025-01-24 21:06:39.001859,Player 4: white,Crewmate,no thought process has been made,Game Time: 0/50\nCurrent phase: Task phase\nIn...,no memory has been processed,- Player 1 moved from the Cafeteria to Medbay....,MOVE from Cafeteria to Upper Engine,"- Since the game is in the task phase, my main..."
4,Game 1,0,2025-01-24 21:06:42.780480,Player 5: black,Crewmate,no thought process has been made,Game Time: 0/50\nCurrent phase: Task phase\nIn...,no memory has been processed,No specific events or actions have been observ...,MOVE from Cafeteria to Upper Engine,As a Crewmate focused on task completion and u...
...,...,...,...,...,...,...,...,...,...,...,...
77,Game 2,5,2025-01-24 21:12:12.065547,Player 3: blue,Crewmate,The current meeting phase involves discussing ...,Game Time: 5/50\nCurrent phase: Meeting phase ...,"- I have completed the tasks ""Fix Wiring"" in A...","- I have completed the tasks ""Fix Wiring"" in A...",VOTE Player 3: blue,- The meeting phase is critical to identifying...
78,Game 2,5,2025-01-24 21:12:16.691521,Player 4: yellow,Impostor,The meeting phase is crucial for manipulating ...,Game Time: 5/50\nCurrent phase: Meeting phase ...,I have successfully completed the task in Admi...,I have successfully completed the task in Admi...,MOVE from Cafeteria to Electrical,"The meeting phase has concluded, and it's cruc..."
79,Game 2,6,2025-01-24 21:12:21.725448,Player 1: red,Crewmate,- The current discussion is centered on task v...,Game Time: 5/50\nCurrent phase: Task phase\nIn...,- The game is currently in the meeting phase. ...,- The current discussion is centered on task v...,"SPEAK: ""Player 3's lack of task completion and...","- The current phase is the task phase, but the..."
80,Game 2,6,2025-01-24 21:12:25.359484,Player 2: purple,Crewmate,"In the meeting phase, Player 1 voted for Playe...",Game Time: 6/50\nCurrent phase: Task phase\nIn...,- I found Player 5 dead in the Cafeteria and r...,"- Player 5 was found dead in the Cafeteria, an...",MOVE from Cafeteria to Weapons,- The collective decision to vote out Player 3...


In [ ]:
# good to go!